In [1]:
# 필요한 라이브러리 import

import folium
import numpy as np
import pandas as pd
from tqdm import tqdm

# 컬럼 전체 확인 가능하도록 출력 범위 설정
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


import geopandas as gpd

#시각화 모듈
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns


# 한글 글자 깨짐 방지
plt.rcParams["font.family"] = 'Nanum Gothic'
sns.set(font="Malgun Gothic", 
        rc={"axes.unicode_minus":False},
        style='darkgrid')


# 불필요한 경고 표시 생략
import warnings
warnings.filterwarnings(action = 'ignore')

/opt/app-root/lib/python3.6/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
# 결측치 확인용 함수

def check_null(data):
    return data.isnull().sum(axis=0)

# Type 1. gid, geometry로 병합할 수 없는 데이터
---
**data1:** 16.대전광역시_기상데이터(2017~2019).csv

In [3]:
data1 = pd.read_csv('16.대전광역시_기상데이터(2017~2019).csv')

## data1 - 날씨 데이터

In [4]:
data1 = pd.read_csv('16.대전광역시_기상데이터(2017~2019).csv')

In [5]:
check_null(data1)

일시                   0
평균기온(°C)             0
최저기온(°C)             0
최고기온(°C)             0
일강수량(mm)           677
최대 풍속(m/s)           0
최대 풍속 풍향(16방위)       0
평균 풍속(m/s)           2
평균 상대습도(%)           0
일 최심적설(cm)        1053
평균 지면온도(°C)          0
안개 계속시간(hr)       1045
dtype: int64

In [6]:
# 날씨 데이터상에 값이 없는 것은 누락된 것으로 판단(도메인 지식) -> 해당 값을 모두 0으로 치환
# data1 = data1.fillna(0)로 처리할 수도 있으나 확실한 처리를 위해 각 컬럼별로 치환한뒤 fillna(0)처리

def fill_empty_to_0(culumn):
    data1[culumn].replace('', 0, inplace=True)
    return data1

In [7]:
has_0val_col = ['일강수량(mm)','평균 풍속(m/s)','일 최심적설(cm)','안개 계속시간(hr)']


for i in has_0val_col:
    fill_empty_to_0(i)

data1 = data1.fillna(0)

In [8]:
# 2번 노트 (Data Processing) 에서 사고일과 병합때 사용하기 위해 일시 -> 사고일로 rename
data1.rename(columns = {'일시' : '사고일'}, inplace = True)

# 전처리 결과 확인
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1095 entries, 0 to 1094
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   사고일             1095 non-null   object 
 1   평균기온(°C)        1095 non-null   float64
 2   최저기온(°C)        1095 non-null   float64
 3   최고기온(°C)        1095 non-null   float64
 4   일강수량(mm)        1095 non-null   float64
 5   최대 풍속(m/s)      1095 non-null   float64
 6   최대 풍속 풍향(16방위)  1095 non-null   int64  
 7   평균 풍속(m/s)      1095 non-null   float64
 8   평균 상대습도(%)      1095 non-null   float64
 9   일 최심적설(cm)      1095 non-null   float64
 10  평균 지면온도(°C)     1095 non-null   float64
 11  안개 계속시간(hr)     1095 non-null   float64
dtypes: float64(10), int64(1), object(1)
memory usage: 102.8+ KB


In [9]:
data1.to_csv('weather_data.csv', index=False )

# Type 2. 사고격자와 gid or geometry 로 병합 가능한 데이터
---
- gid or geometry 기준으로 병합


**data2:** = 12.대전광역시_인구정보(총인구) 

**data3:** =  13.대전광역시_인구정보(고령) 

**data4:** =  14.대전광역시_인구정보(생산가능)  

**data5:** =  15.대전광역시_인구정보(유소년)  

**data6:** =  20.대전광역시_평일_일별_시간대별_추정교통량(2018) 

**data7:** =  21.대전광역시_평일_일별_혼잡빈도강도(2018) 

**data8:** =  22.대전광역시_평일_일별_혼잡시간강도(2018) 

In [10]:
data2 = gpd.read_file('12.대전광역시_인구정보(총인구).geojson')
data3 = gpd.read_file('13.대전광역시_인구정보(고령).geojson')
data4 = gpd.read_file('14.대전광역시_인구정보(생산가능).geojson')
data5 = gpd.read_file('15.대전광역시_인구정보(유소년).geojson')
data6 = gpd.read_file('18.대전광역시_교통노드(2018).geojson')
data7 = pd.read_csv('20.대전광역시_평일_일별_시간대별_추정교통량(2018).csv')
data8 = pd.read_csv('21.대전광역시_평일_일별_혼잡빈도강도(2018).csv')
data9 = pd.read_csv('22.대전광역시_평일_일별_혼잡시간강도(2018).csv')

In [11]:
# 인구 데이터 전처리용 함수 생성
# val에 있는 nan값 전처리 -> 거주 인구가 없는 지역이므로 0으로 처리(도메인 지식)

def convert_null_zero(tabel):
    isnull_sum = tabel.isnull().sum(axis=0)
    col_name = []
    
    for i, z in enumerate(isnull_sum.values):
        if z != 0:
            print("{0}:     {1}".format(isnull_sum.keys()[i], isnull_sum[i]))
            col_name.append(isnull_sum.keys()[i])
            
    tabel[col_name] = tabel[col_name].fillna(0)
    # 전처리 결과 출력
    print(tabel.info())
    return tabel

# 데이터  2 : 총 인구 데이터
|Col|Meaning|
|:-|:-|
|gid|격자 고유 번호|
|val|해당 격자당 인구 수|
|geometry|격자데이터|

- 대전시 전체 격자를 기준으로 한 것이기 때문에 거주인구가 없는 격자는 NaN값이 발생할 수 있다.

In [12]:
data2.head()

,gid,val,geometry
0,다마846996,NaN,"MULTIPOLYGON (((127.32871 36.19342, 127.32871 ..."
1,다마847995,NaN,"MULTIPOLYGON (((127.32983 36.19252, 127.32982 ..."
2,다마847996,NaN,"MULTIPOLYGON (((127.32982 36.19342, 127.32982 ..."
3,다마847997,NaN,"MULTIPOLYGON (((127.32982 36.19432, 127.32982 ..."
4,다마847998,NaN,"MULTIPOLYGON (((127.32982 36.19522, 127.32982 ..."


In [13]:
check_null(data2)

gid             0
val         43803
geometry        0
dtype: int64

In [14]:
data2 = convert_null_zero(data2)

val:     43803
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 54912 entries, 0 to 54911
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   gid       54912 non-null  object  
 1   val       54912 non-null  float64 
 2   geometry  54912 non-null  geometry
dtypes: float64(1), geometry(1), object(1)
memory usage: 1.3+ MB
None


In [15]:
# 추가 전처리 위해 파일로 저장
data2.to_file('total_population.geojson', driver= 'GeoJSON')

# 데이터 3 : 고령자 데이터 

|Col|Meaning|
|:-|:-|
|gid|격자 고유 번호|
|val|해당 격자당 고령자 인구 수|
|geometry|격자데이터|

- 대전시 전체 격자를 기준으로 한 것이기 때문에 거주인구가 없는 격자는 NaN값이 발생할 수 있다.

In [16]:
data3.head()

,gid,val,geometry
0,다마846996,NaN,"MULTIPOLYGON (((127.32871 36.19342, 127.32871 ..."
1,다마847995,NaN,"MULTIPOLYGON (((127.32983 36.19252, 127.32982 ..."
2,다마847996,NaN,"MULTIPOLYGON (((127.32982 36.19342, 127.32982 ..."
3,다마847997,NaN,"MULTIPOLYGON (((127.32982 36.19432, 127.32982 ..."
4,다마847998,NaN,"MULTIPOLYGON (((127.32982 36.19522, 127.32982 ..."


In [17]:
check_null(data3)

gid             0
val         45238
geometry        0
dtype: int64

In [18]:
data3 = convert_null_zero(data3)

val:     45238
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 54912 entries, 0 to 54911
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   gid       54912 non-null  object  
 1   val       54912 non-null  float64 
 2   geometry  54912 non-null  geometry
dtypes: float64(1), geometry(1), object(1)
memory usage: 1.3+ MB
None


In [19]:
data3.to_file('old_population.geojson', driver= 'GeoJSON')

# 데이터 4 : 생산가능 인구

|Col|Meaning|
|:-|:-|
|gid|격자 고유 번호|
|val|해당 격자당 생산가능 인구 수|
|geometry|격자데이터|

- 대전시 전체 격자를 기준으로 한 것이기 때문에 거주인구가 없는 격자는 NaN값이 발생할 수 있다.

In [20]:
data4.head()

,gid,val,geometry
0,다마846996,NaN,"MULTIPOLYGON (((127.32871 36.19342, 127.32871 ..."
1,다마847995,NaN,"MULTIPOLYGON (((127.32983 36.19252, 127.32982 ..."
2,다마847996,NaN,"MULTIPOLYGON (((127.32982 36.19342, 127.32982 ..."
3,다마847997,NaN,"MULTIPOLYGON (((127.32982 36.19432, 127.32982 ..."
4,다마847998,NaN,"MULTIPOLYGON (((127.32982 36.19522, 127.32982 ..."


In [21]:
check_null(data4)

gid             0
val         44359
geometry        0
dtype: int64

In [22]:
data4 = convert_null_zero(data4)

val:     44359
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 54912 entries, 0 to 54911
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   gid       54912 non-null  object  
 1   val       54912 non-null  float64 
 2   geometry  54912 non-null  geometry
dtypes: float64(1), geometry(1), object(1)
memory usage: 1.3+ MB
None


In [23]:
data4.to_file('labor_population.geojson', driver= 'GeoJSON')

# 데이터 5 : 유소년 인구

|Col|Meaning|
|:-|:-|
|gid|격자 고유 번호|
|val|해당 격자당 유소년 인구 수|
|geometry|격자데이터|

- 대전시 전체 격자를 기준으로 한 것이기 때문에 거주인구가 없는 격자는 NaN값이 발생할 수 있다.

In [24]:
data5.head()

,gid,val,geometry
0,다마846996,NaN,"MULTIPOLYGON (((127.32871 36.19342, 127.32871 ..."
1,다마847995,NaN,"MULTIPOLYGON (((127.32983 36.19252, 127.32982 ..."
2,다마847996,NaN,"MULTIPOLYGON (((127.32982 36.19342, 127.32982 ..."
3,다마847997,NaN,"MULTIPOLYGON (((127.32982 36.19432, 127.32982 ..."
4,다마847998,NaN,"MULTIPOLYGON (((127.32982 36.19522, 127.32982 ..."


In [25]:
check_null(data5)

gid             0
val         48086
geometry        0
dtype: int64

In [26]:
data5 = convert_null_zero(data5)

val:     48086
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 54912 entries, 0 to 54911
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   gid       54912 non-null  object  
 1   val       54912 non-null  float64 
 2   geometry  54912 non-null  geometry
dtypes: float64(1), geometry(1), object(1)
memory usage: 1.3+ MB
None


In [27]:
data5.to_file('youth_population.geojson', driver= 'GeoJSON')

# 데이터 6 : 교통노드 
|Col|Meaning|
|:-|:-|
|NODE_ID|노드 ID값|
|NODE_TYPE|노드의 유형코드 번호 : 7개 유형 (교차로, 행정경계, 도로시설물 등)|
|NODE_NAME|노드의 이름|
|TURN_P|회전제한 유무 코드 : 회전금지구간(1)|
|REMAKR|노드관련 비고|
|geometry|MULTIPOLYGON 데이터|

## 데이터 6-1 : 유턴금지구역

In [28]:
check_null(data6)

NODE_ID         0
NODE_TYPE       0
NODE_NAME       0
TURN_P          0
REMARK       1836
geometry        0
dtype: int64

In [29]:
turn_p =  data6[data6['TURN_P']=='1'] #유턴금지구역
turn_p

,NODE_ID,NODE_TYPE,NODE_NAME,TURN_P,REMARK,geometry
10,1850000900,101,진잠네거리,1,None,MULTIPOINT (127.32363 36.29869)
12,1850001300,101,-,1,None,MULTIPOINT (127.33709 36.30155)
13,1850001400,101,건양대네거리,1,None,MULTIPOINT (127.34046 36.30252)
14,1850001500,101,계룡아파트삼거리,1,None,MULTIPOINT (127.34935 36.30423)
15,1850001700,101,가수원네거리,1,None,MULTIPOINT (127.35327 36.30512)
...,...,...,...,...,...,...
2627,1850075200,101,원정동입구,1,좌회전금지,MULTIPOINT (127.33194 36.24935)
2630,1850076200,101,기성초등학교입구,1,비보호좌,MULTIPOINT (127.33618 36.25065)
2631,1850076300,101,승암골입구,1,비보호,MULTIPOINT (127.33880 36.25352)
2633,1850077200,101,사진개정류장앞,1,비보호좌,MULTIPOINT (127.34582 36.26184)


In [30]:
check_null(data6)

NODE_ID         0
NODE_TYPE       0
NODE_NAME       0
TURN_P          0
REMARK       1836
geometry        0
dtype: int64

In [31]:
turn_p.to_file('turn_p.geojson', driver= 'GeoJSON')

## 데이터 6-2 : 교차로

In [32]:
instersections = data6[data6['NODE_TYPE']=='101'] # 교차로
instersections

,NODE_ID,NODE_TYPE,NODE_NAME,TURN_P,REMARK,geometry
0,1860000303,101,두마IC,0,None,MULTIPOINT (127.26479 36.27539)
1,1850000703,101,서대전IC,0,None,MULTIPOINT (127.31957 36.28751)
2,1860000304,101,삿갓집입구,0,None,MULTIPOINT (127.28988 36.26759)
3,1860000602,101,방동대교앞,0,None,MULTIPOINT (127.30941 36.28333)
4,1860000501,101,방동사거리,0,None,MULTIPOINT (127.30034 36.28004)
...,...,...,...,...,...,...
2629,1850075800,101,교차로,0,None,MULTIPOINT (127.33395 36.24969)
2630,1850076200,101,기성초등학교입구,1,비보호좌,MULTIPOINT (127.33618 36.25065)
2631,1850076300,101,승암골입구,1,비보호,MULTIPOINT (127.33880 36.25352)
2633,1850077200,101,사진개정류장앞,1,비보호좌,MULTIPOINT (127.34582 36.26184)


In [33]:
check_null(data6)

NODE_ID         0
NODE_TYPE       0
NODE_NAME       0
TURN_P          0
REMARK       1836
geometry        0
dtype: int64

In [34]:
instersections.to_file('instersections.geojson', driver= 'GeoJSON')

# 데이터 7 : 시간대별 추정교통량
|Col|Meaning|
|:-|:-|
|상세도로망_LinkID|해당 링크의 ID ( 총 9자리, 맨마지막 2자리는 상행 & 하행을 구분)|
|도로등급|해당 도로의 등급|
|링크길이|해당 도로의 길이|
|도로명|해당 도로명|
|시도명|해당 시도명|
|시군구명|해당 시군구명|
|읍면동명|해당 읍면동명|
|시간적범위|측정에 대한 시간적 범위 정보 ( fulltime, 1 ~ 23 으로 각 시간단위)|
|전체_추정교통량|전체 교통량 추정치|
|승용차_추정교통량|승용차의 교통량 추정치|
|버스_추정교통량|버스의 교통량 추정치|
|화물차_추정교통량|화물차의 교통량 추정치|

In [35]:
check_null(data6)

NODE_ID         0
NODE_TYPE       0
NODE_NAME       0
TURN_P          0
REMARK       1836
geometry        0
dtype: int64

In [36]:
# 도로망 링크 전처리용 함수 생성

def For_slicing(tabel):
    tabel['상세도로망_LinkID'] = tabel['상세도로망_LinkID'].astype(str) #slicing을 위해 형변환
    tabel['상세도로망_LinkID'] = tabel['상세도로망_LinkID'].str[:-2] # 상하행 구분이 필요없으므로 앞의 7자리로 도로만 구분하도록 변경
    tabel['상세도로망_LinkID'] = tabel['상세도로망_LinkID'].astype(str) #slicing을 위해 형변환
    return tabel

In [37]:
data7 = For_slicing(data7)

In [38]:
# 시간적 범위는 범위값은 1 ~ 23인데 '' 이 붙은 값도 있고, '' 이없는 값도 있음
# 해당 데이터에서 중요하게 보고싶은 것은 '교통량' 이므로 시간대를 fulltime으로 통일
# 시간대의 차이에서 오는 데이터 불균형은 group by 를 mean으로 하여 전체 평균 으로 치환할 예정
data7 = data7[data7['시간적범위']=='fulltime']

In [39]:
# 전체 시간 (24시간 대비 평균 교통량 계산)
data7 = data7[['상세도로망_LinkID','전체_추정교통량','승용차_추정교통량', '버스_추정교통량', '화물차_추정교통량']].groupby(['상세도로망_LinkID']).mean()
data7.reset_index(inplace=True)

In [40]:
data7.head(2)

,상세도로망_LinkID,전체_추정교통량,승용차_추정교통량,버스_추정교통량,화물차_추정교통량
0,553700043,695.0,458.0,19.0,218.0
1,553700273,3871.5,3449.0,22.0,401.0


In [41]:
check_null(data7)

상세도로망_LinkID    0
전체_추정교통량        0
승용차_추정교통량       0
버스_추정교통량        0
화물차_추정교통량       0
dtype: int64

In [42]:
data7.to_csv('volume_of_traffic.csv',index=False)

# 데이터 8 : 평일 일별 혼잡 빈도 강도
---

|Col|Meaning|
|:-|:-|
|상세도로망_LinkID|	해당 링크의 ID|
|도로등급|	해당 도로의 등급|
|링크길이|	해당 도로의 길이|
|도로명|	해당 도로명|
|시도명|	해당 시도명|
|시군구명| 	해당 시군구명|
|읍면동명|	해당 읍면동명|
|혼잡빈도강도|해당 지점의 혼잡빈도 강도 (0.0 ~ 100 사이)|

In [43]:
data8.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18044 entries, 0 to 18043
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   상세도로망_LinkID  18044 non-null  int64  
 1   도로등급          18044 non-null  int64  
 2   링크길이          18044 non-null  float64
 3   도로명           15999 non-null  object 
 4   시도명           18044 non-null  object 
 5   시군구명          18044 non-null  object 
 6   읍면동명          18044 non-null  object 
 7   혼잡빈도강도        18044 non-null  float64
dtypes: float64(2), int64(2), object(4)
memory usage: 1.1+ MB


In [44]:
check_null(data8)

상세도로망_LinkID       0
도로등급               0
링크길이               0
도로명             2045
시도명                0
시군구명               0
읍면동명               0
혼잡빈도강도             0
dtype: int64

In [45]:
data8 = For_slicing(data8)

data8 = data8[['상세도로망_LinkID','혼잡빈도강도']].groupby(['상세도로망_LinkID']).mean()
data8.reset_index(inplace=True)

In [46]:
data8.head(2)

,상세도로망_LinkID,혼잡빈도강도
0,553700273,5.340
1,553700308,1.705


In [47]:
data8.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10134 entries, 0 to 10133
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   상세도로망_LinkID  10134 non-null  object 
 1   혼잡빈도강도        10134 non-null  float64
dtypes: float64(1), object(1)
memory usage: 158.5+ KB


In [48]:
data8.to_csv('honjap_frequency.csv', index=False)

#  데이터 9 : 평일 일별 혼잡 시간 강도
---

|Col|Meaning|
|:-|:-|
|상세도로망_LinkID|	해당 링크의 ID|
|도로등급|	해당 도로의 등급|
|링크길이|	해당 도로의 길이|
|도로명|	해당 도로명|
|시도명|	해당 시도명|
|시군구명| 	해당 시군구명|
|읍면동명|	해당 읍면동명|
|혼잡빈도강도|해당 지점의 혼잡빈도 강도 (0.0 ~ 100 사이)|

In [49]:
data9.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18044 entries, 0 to 18043
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   상세도로망_LinkID  18044 non-null  int64  
 1   도로등급          18044 non-null  int64  
 2   링크길이          18044 non-null  float64
 3   도로명           15999 non-null  object 
 4   시도명           18044 non-null  object 
 5   시군구명          18044 non-null  object 
 6   읍면동명          18044 non-null  object 
 7   혼잡시간강도        18044 non-null  float64
dtypes: float64(2), int64(2), object(4)
memory usage: 1.1+ MB


In [50]:
check_null(data9)

상세도로망_LinkID       0
도로등급               0
링크길이               0
도로명             2045
시도명                0
시군구명               0
읍면동명               0
혼잡시간강도             0
dtype: int64

In [51]:
data9 = For_slicing(data9)

data9 = data9[['상세도로망_LinkID','혼잡시간강도']].groupby(['상세도로망_LinkID']).mean()
data9.reset_index(inplace=True)

In [52]:
data9.head(2)

,상세도로망_LinkID,혼잡시간강도
0,553700273,15.25
1,553700308,4.54


In [53]:
data9.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10134 entries, 0 to 10133
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   상세도로망_LinkID  10134 non-null  object 
 1   혼잡시간강도        10134 non-null  float64
dtypes: float64(1), object(1)
memory usage: 158.5+ KB


In [54]:
data9.to_csv('honjap_intensity.csv',index=False)